# Airline AI Assistant

In [1]:
# Imports 
import os
import json
from dotenv import load_dotenv
from openai import OpenAI
import gradio as gr

In [2]:
# Initializing api_key, model, base_url, system_prompt
load_dotenv()
ModelConfig = {
    "gemini":{
        "api_key":os.getenv('GOOGLE_API_KEY'),
        "base_url":"https://generativelanguage.googleapis.com/v1beta/openai/",
        "model":'gemini-2.0-flash'
    },
    "sambanova":{
        'api_key':os.getenv('SAMBANOVA_API_KEY'),
        'base_url':"https://api.sambanova.ai/v1",
        'model':"Llama-4-Maverick-17B-128E-Instruct"
    }
}

gemini_client = OpenAI(
    api_key=ModelConfig['gemini']['api_key'],
    base_url=ModelConfig['gemini']['base_url']
)

sambanova_client = OpenAI(
    api_key=ModelConfig['sambanova']['api_key'],
    base_url=ModelConfig['sambanova']['base_url']
)

# system prompt
sys_prompt = "You are a helpful assistant for an Airline called flightAI."
sys_prompt += "Give short, courteous answer, no more than one sentence. Always be accurate. If you don't know the answer, say so."

In [3]:
def chat(message, history):
    # Convert history tuples to the correct format
    formatted_history = []
    for human, ai in history:
        formatted_history.append({"role": "user", "content": human})
        formatted_history.append({"role": "assistant", "content": ai})
    
    # Add system message and current message
    messages = [{"role": "system", "content": sys_prompt}] + formatted_history + [{"role": "user", "content": message}]

    response = gemini_client.chat.completions.create(
        model=ModelConfig['gemini']['model'],
        messages=messages
    )

    return response.choices[0].message.content

In [4]:
view = gr.ChatInterface(fn=chat).launch(share=True)

c:\Users\user\Desktop\Git-hub\LLM-engieering\venv\Lib\site-packages\gradio\chat_interface.py:339: UserWarning: The 'tuples' format for chatbot messages is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style 'role' and 'content' keys.
  self.chatbot = Chatbot(


* Running on local URL:  http://127.0.0.1:7860
* Running on public URL: https://06b7222e7773d93f69.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


## Tools 

In [5]:
# Create a simple function to find the price for the ticket 
ticket_price = {'london': "$799", 'paris': "$899", 'berlin': "$599"}

def get_ticket_price(destination_city):
    print(f"Tool get_ticket_price called for {destination_city}")
    city = destination_city.lower()
    return ticket_price.get(city, 'Unknown')

In [6]:
get_ticket_price('BERLIN')

Tool get_ticket_price called for BERLIN


'$599'

In [1]:
# Now we will create a tool, but follows some particular structure
price_fuction = {
    "name": "get_ticket_price",
    "description": "Get the price for the return ticket to the destination city. Call this whenever you need to know the ticket price, for example: when customers ask 'how much is a ticket to this city'",
    "parameters": {
        "type": "object",
        "properties": {
            "destination_city": {
                "type": "string",
                "description": "The city that the customer wants to travel to",
            },
        },
        "required": ['destination_city'],
        "additionalProperties": False
    }
}

In [14]:
tools = [{"type": "function", 'function': price_fuction}]

In [12]:
## Getting OpenAI to use our tool
# There's some fiddly stuff to allow OpenAI "to call our tool"
# What we actually do is give the LLM the opportunity  to inform us that it wants us to run the tool.
# Here's how the new chat function looks:

In [15]:
# We have to write that function handle_tool_call:

def handle_tool_call(message):
    tool_call = message.tool_calls[0]
    arguments = json.loads(tool_call.function.arguments)
    city = arguments.get('destination_city')
    price = get_ticket_price(city)
    response = {
        "role": "tool",
        "content": json.dumps({"destination_city": city,"price": price}),
        "tool_call_id": tool_call.id
    }
    return response, city

In [22]:
def chat(message, history):
    messages = [{'role':'system', 'content':sys_prompt}]
    for human, assistant in history:
        message.

In [23]:
gr.ChatInterface(fn=chat, type="messages").launch()

* Running on local URL:  http://127.0.0.1:7864
* To create a public link, set `share=True` in `launch()`.


Tool get_ticket_price called for berlin
